"The spectral centroid indicates at which frequency the energy of a spectrum is centred upon" (Chauhan, 2020)

Racharla et. al (2020) used SVM classifiers with a high success (79% accuracy) in instrument recognition. They used input features such as MFCC, Zero-crossing rate, Spectral centroid, and others using Librosa.

Breebaart (2004) compared various features such as mel-frequency cepstral coefficients (MFCC), auditory filter-bank temporal envelope (AFTE), standard low-level (SLL) and psychoacoustic (PA) features when training a standard Gaussian framework for both genre and general audio classification.

**AFTE features**:
GammaTone filter banks.

**SLL features**:
Root-mean-square (RMS) level, spectral (centroid, bandwidth, and roll-off), zero-crossing rate, band energy ratio, delta spectrum magnitude, and pitch.

**PA features**:
Roughness (the perception of temporal envelope modulations in the range of about 20-150 Hz), loudness (the sensation of signal strength) and sharpness (the spectral density and the relative strength of high-frequency energy).

It was demonstrated that AFTE features provided the highest accuracy in Breebaart's study however, Deng (2008) showed that MFCC and perception-based features dominated in SVM instrument classification models.

## Convert .bib to Harvard
Zaman, K. et al. (2023) ‘A Survey of Audio Classification Using Deep Learning’, IEEE Access, 11, pp. 106620–106649. doi: 10.1109/ACCESS.2023.3318015.


Racharla, K. et al. (2020) ‘Predominant Musical Instrument Classification based on Spectral Features’, in 2020 7th International Conference on Signal Processing and Integrated Networks (SPIN). IEEE. doi: 10.1109/spin48934.2020.9071125.

